In [145]:
import re

import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
def fillCityFuelNullBodyType(df:pd.DataFrame, cat_column: str, num_column: str, medianList: pd.Series) -> pd.DataFrame:
    for x in df[cat_column].unique():
        mask = (df[cat_column]==x)&(df[num_column].isna())
        df.loc[mask, num_column] = df.loc[mask, num_column].fillna(medianList[x])
    return df

In [3]:
def enconding(df: pd.DataFrame, column: str) -> pd.DataFrame:
    df.reset_index(drop=True, inplace=True)
    ohe = OneHotEncoder().fit(df[[column]])
    ohe_transform = ohe.transform(df[[column]])

    df = pd.concat(objs=[df, pd.DataFrame(data=ohe_transform.toarray(), columns=ohe.categories_[0])], axis=1)
    return df.drop(columns=[column])

In [4]:
df = pd.read_csv(filepath_or_buffer="used_cars_data.csv")

C:\Users\gustavo.o.gois\AppData\Local\Temp\ipykernel_13392\2742004424.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath_or_buffer="used_cars_data.csv")


In [5]:
df.head()

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [6]:
df["bed"].isna().value_counts()

True     2980472
False      19568
Name: bed, dtype: int64

In [7]:
for x in df.columns:
    print("{0}".format(type(df[x][0])))

<class 'str'>
<class 'str'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'float'>
<class 'numpy.bool_'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'str'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.bool_'>
<class 'float'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'str'>
<class '

In [8]:
df.drop(columns=["vin"], inplace=True)

In [9]:
df["back_legroom"] = df["back_legroom"].str.replace(pat=" in", repl="")
df["back_legroom"] = df["back_legroom"].str.replace(pat="--", repl="0")
df["back_legroom"] = df["back_legroom"].astype(dtype="float16")
df.head()

,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,35.09375,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,522,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,38.09375,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,207,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,35.40625,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,1233,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,37.59375,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,196,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,38.09375,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,137,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [10]:
df["bed"].fillna(value="N/A", inplace=True)
df.head()

,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,35.09375,N/A,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,522,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,38.09375,N/A,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,207,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,35.40625,N/A,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,1233,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,37.59375,N/A,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,196,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,38.09375,N/A,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,137,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [11]:
df["bed_height"].unique()

array([nan, '--'], dtype=object)

In [12]:
df.drop(columns=["bed_length", "bed_height"], inplace=True)

In [13]:
df["body_type"].unique()

array(['SUV / Crossover', 'Sedan', 'Coupe', 'Hatchback', 'Pickup Truck',
       'Wagon', 'Minivan', 'Van', 'Convertible', nan], dtype=object)

In [14]:
df["body_type"].value_counts(dropna=False)

SUV / Crossover    1416402
Sedan               742036
Pickup Truck        474595
Hatchback            88374
Minivan              79802
Coupe                71607
Van                  47166
Wagon                40505
Convertible          26010
NaN                  13543
Name: body_type, dtype: int64

In [15]:
df["cabin"].value_counts(dropna=False)

NaN               2936507
Crew Cab            51083
Extended Cab         7960
Regular Cab          2966
Large Crew Cab       1524
Name: cabin, dtype: int64

In [16]:
len(df[(df.body_type.isna()) & (df.cabin.isna())])

13486

In [17]:
df.dropna(subset=["body_type"], inplace=True)
df.head()

,back_legroom,bed,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,35.09375,N/A,SUV / Crossover,NaN,Bayamon,NaN,NaN,522,960,[!@@Additional Info@@!]Engine: 2.4L I4 ZERO EV...,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,38.09375,N/A,SUV / Crossover,NaN,San Juan,NaN,NaN,207,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,35.40625,N/A,Sedan,NaN,Guaynabo,17.0,NaN,1233,969,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,37.59375,N/A,SUV / Crossover,NaN,San Juan,NaN,NaN,196,922,"[!@@Additional Info@@!]Fog Lights,7 Seat Packa...",...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,38.09375,N/A,SUV / Crossover,NaN,San Juan,NaN,NaN,137,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [18]:
df[df.cabin.isna()]["body_type"].value_counts(dropna=False)

SUV / Crossover    1416383
Sedan               742036
Pickup Truck        411162
Hatchback            88374
Minivan              79802
Coupe                71606
Van                  47164
Wagon                40505
Convertible          25989
Name: body_type, dtype: int64

In [19]:
(df.groupby(["body_type"])["cabin"].value_counts(dropna=False))

body_type        cabin         
Convertible      NaN                 25989
                 Regular Cab            21
Coupe            NaN                 71606
                 Regular Cab             1
Hatchback        NaN                 88374
Minivan          NaN                 79802
Pickup Truck     NaN                411162
                 Crew Cab            51045
                 Extended Cab         7942
                 Regular Cab          2922
                 Large Crew Cab       1524
SUV / Crossover  NaN               1416383
                 Crew Cab               19
Sedan            NaN                742036
Van              NaN                 47164
                 Extended Cab            2
Wagon            NaN                 40505
Name: cabin, dtype: int64

In [20]:
df.drop(columns=["cabin"], inplace=True)
df.head()

,back_legroom,bed,body_type,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,35.09375,N/A,SUV / Crossover,Bayamon,NaN,NaN,522,960,[!@@Additional Info@@!]Engine: 2.4L I4 ZERO EV...,I4,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,38.09375,N/A,SUV / Crossover,San Juan,NaN,NaN,207,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,35.40625,N/A,Sedan,Guaynabo,17.0,NaN,1233,969,NaN,H4,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,37.59375,N/A,SUV / Crossover,San Juan,NaN,NaN,196,922,"[!@@Additional Info@@!]Fog Lights,7 Seat Packa...",V6,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,38.09375,N/A,SUV / Crossover,San Juan,NaN,NaN,137,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [21]:
df["city"].value_counts(dropna=False)

Houston         43681
San Antonio     24830
Columbus        18230
Miami           18074
Jacksonville    16764
                ...  
Trafalgar           1
Lovington           1
Newmarket           1
Wilmerding          1
Bishop              1
Name: city, Length: 4687, dtype: int64

In [22]:
len(df[df.city_fuel_economy.isna()]) #combine_fuel_economy

477771

In [23]:
dfLenght = len(df)
dfLenght

2986497

In [24]:
# Percentage of city_fuel_economy as nan
len(df[df.city_fuel_economy.isna()])/dfLenght

0.15997705673235232

In [25]:
df.columns

Index(['back_legroom', 'bed', 'body_type', 'city', 'city_fuel_economy',
       'combine_fuel_economy', 'daysonmarket', 'dealer_zip', 'description',
       'engine_cylinders', 'engine_displacement', 'engine_type',
       'exterior_color', 'fleet', 'frame_damaged', 'franchise_dealer',
       'franchise_make', 'front_legroom', 'fuel_tank_volume', 'fuel_type',
       'has_accidents', 'height', 'highway_fuel_economy', 'horsepower',
       'interior_color', 'isCab', 'is_certified', 'is_cpo', 'is_new',
       'is_oemcpo', 'latitude', 'length', 'listed_date', 'listing_color',
       'listing_id', 'longitude', 'main_picture_url', 'major_options',
       'make_name', 'maximum_seating', 'mileage', 'model_name', 'owner_count',
       'power', 'price', 'salvage', 'savings_amount', 'seller_rating', 'sp_id',
       'sp_name', 'theft_title', 'torque', 'transmission',
       'transmission_display', 'trimId', 'trim_name',
       'vehicle_damage_category', 'wheel_system', 'wheel_system_display',
       '

In [26]:
df["city_fuel_economy"].value_counts(dropna=False)

NaN      477771
19.0     198958
18.0     191822
22.0     174741
21.0     170910
          ...  
78.0          6
98.0          5
93.0          3
112.0         3
7.0           2
Name: city_fuel_economy, Length: 101, dtype: int64

In [27]:
fuelMed = (df.groupby(["body_type"]).mean())["city_fuel_economy"]
fuelMed

C:\Users\gustavo.o.gois\AppData\Local\Temp\ipykernel_13392\3938099843.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fuelMed = (df.groupby(["body_type"]).mean())["city_fuel_economy"]


body_type
Convertible        18.800167
Coupe              19.132341
Hatchback          39.284222
Minivan            17.803321
Pickup Truck       16.530865
SUV / Crossover    21.629598
Sedan              26.347504
Van                20.603778
Wagon              32.393020
Name: city_fuel_economy, dtype: float64

In [28]:
df["city_fuel_economy"].mean()

22.693242307051467

In [29]:
df = fillCityFuelNullBodyType(df=df, cat_column="body_type", num_column="city_fuel_economy", medianList=fuelMed)
df.head()

,back_legroom,bed,body_type,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,35.09375,N/A,SUV / Crossover,Bayamon,21.629598,NaN,522,960,[!@@Additional Info@@!]Engine: 2.4L I4 ZERO EV...,I4,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,38.09375,N/A,SUV / Crossover,San Juan,21.629598,NaN,207,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,35.40625,N/A,Sedan,Guaynabo,17.000000,NaN,1233,969,NaN,H4,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,37.59375,N/A,SUV / Crossover,San Juan,21.629598,NaN,196,922,"[!@@Additional Info@@!]Fog Lights,7 Seat Packa...",V6,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,38.09375,N/A,SUV / Crossover,San Juan,21.629598,NaN,137,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [30]:
df["combine_fuel_economy"].value_counts(dropna=False)

NaN    2986497
Name: combine_fuel_economy, dtype: int64

In [31]:
df.drop(columns=["combine_fuel_economy"], inplace=True)

In [32]:
df.loc[df.daysonmarket.isnull()]

,back_legroom,bed,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year


In [33]:
df.loc[df.dealer_zip.isna()]

,back_legroom,bed,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year


In [34]:
df.drop(columns=["description"], inplace=True)

In [35]:
df.head()

,back_legroom,bed,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,engine_type,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,35.09375,N/A,SUV / Crossover,Bayamon,21.629598,522,960,I4,1300.0,I4,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,38.09375,N/A,SUV / Crossover,San Juan,21.629598,207,922,I4,2000.0,I4,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,35.40625,N/A,Sedan,Guaynabo,17.000000,1233,969,H4,2500.0,H4,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,37.59375,N/A,SUV / Crossover,San Juan,21.629598,196,922,V6,3000.0,V6,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,38.09375,N/A,SUV / Crossover,San Juan,21.629598,137,922,I4,2000.0,I4,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [36]:
df.columns

Index(['back_legroom', 'bed', 'body_type', 'city', 'city_fuel_economy',
       'daysonmarket', 'dealer_zip', 'engine_cylinders', 'engine_displacement',
       'engine_type', 'exterior_color', 'fleet', 'frame_damaged',
       'franchise_dealer', 'franchise_make', 'front_legroom',
       'fuel_tank_volume', 'fuel_type', 'has_accidents', 'height',
       'highway_fuel_economy', 'horsepower', 'interior_color', 'isCab',
       'is_certified', 'is_cpo', 'is_new', 'is_oemcpo', 'latitude', 'length',
       'listed_date', 'listing_color', 'listing_id', 'longitude',
       'main_picture_url', 'major_options', 'make_name', 'maximum_seating',
       'mileage', 'model_name', 'owner_count', 'power', 'price', 'salvage',
       'savings_amount', 'seller_rating', 'sp_id', 'sp_name', 'theft_title',
       'torque', 'transmission', 'transmission_display', 'trimId', 'trim_name',
       'vehicle_damage_category', 'wheel_system', 'wheel_system_display',
       'wheelbase', 'width', 'year'],
      dtype='obj

In [37]:
df["engine_cylinders"].value_counts(dropna=False)

I4                           1413434
V6                            736076
V8                            279692
NaN                            94977
V8 Flex Fuel Vehicle           78134
I4 Hybrid                      72393
V6 Flex Fuel Vehicle           68222
H4                             64934
I3                             54301
I6                             28258
I6 Diesel                      23433
V8 Biodiesel                   21990
I4 Flex Fuel Vehicle            9187
I4 Diesel                       7210
V6 Diesel                       6657
V8 Diesel                       6254
I5                              5299
H6                              4686
V6 Biodiesel                    3611
V6 Hybrid                       3058
V12                             1316
V10                             1287
I2                               897
W12                              484
V8 Hybrid                        127
W12 Flex Fuel Vehicle             97
I5 Biodiesel                      95
V

In [38]:
df["engine_displacement"].value_counts(dropna=False)

2000.0    521314
3500.0    328845
2500.0    326083
3600.0    270507
2400.0    194332
           ...  
8400.0        41
8300.0        38
4100.0        37
7400.0        19
7500.0         4
Name: engine_displacement, Length: 68, dtype: int64

In [39]:
df["engine_type"].value_counts(dropna=False)

I4                           1413434
V6                            736076
V8                            279692
NaN                            94977
V8 Flex Fuel Vehicle           78134
I4 Hybrid                      72393
V6 Flex Fuel Vehicle           68222
H4                             64934
I3                             54301
I6                             28258
I6 Diesel                      23433
V8 Biodiesel                   21990
I4 Flex Fuel Vehicle            9187
I4 Diesel                       7210
V6 Diesel                       6657
V8 Diesel                       6254
I5                              5299
H6                              4686
V6 Biodiesel                    3611
V6 Hybrid                       3058
V12                             1316
V10                             1287
I2                               897
W12                              484
V8 Hybrid                        127
W12 Flex Fuel Vehicle             97
I5 Biodiesel                      95
V

In [40]:
df["engine_cylinders"].equals(df["engine_type"])

True

In [41]:
df.drop(columns=["engine_cylinders"], inplace=True)

In [42]:
df.head()

,back_legroom,bed,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,engine_type,exterior_color,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,35.09375,N/A,SUV / Crossover,Bayamon,21.629598,522,960,1300.0,I4,Solar Yellow,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,38.09375,N/A,SUV / Crossover,San Juan,21.629598,207,922,2000.0,I4,Narvik Black,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,35.40625,N/A,Sedan,Guaynabo,17.000000,1233,969,2500.0,H4,None,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,37.59375,N/A,SUV / Crossover,San Juan,21.629598,196,922,3000.0,V6,Eiger Gray,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,38.09375,N/A,SUV / Crossover,San Juan,21.629598,137,922,2000.0,I4,Narvik Black,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [43]:
df.dropna(subset=["engine_type"], inplace=True)
df["engine_type"].value_counts(dropna=False)

I4                           1413434
V6                            736076
V8                            279692
V8 Flex Fuel Vehicle           78134
I4 Hybrid                      72393
V6 Flex Fuel Vehicle           68222
H4                             64934
I3                             54301
I6                             28258
I6 Diesel                      23433
V8 Biodiesel                   21990
I4 Flex Fuel Vehicle            9187
I4 Diesel                       7210
V6 Diesel                       6657
V8 Diesel                       6254
I5                              5299
H6                              4686
V6 Biodiesel                    3611
V6 Hybrid                       3058
V12                             1316
V10                             1287
I2                               897
W12                              484
V8 Hybrid                        127
W12 Flex Fuel Vehicle             97
I5 Biodiesel                      95
V8 Compressed Natural Gas         95
H

In [44]:
# df["engine_type"].
# re.search(pattern="^[A-Z0-9]", string=df["engine_type"])
df["engine"] = df["engine_type"].apply(lambda x: re.search(pattern="^[A-Z][0-9]", string=x).group(0))
df["engine"].unique()

array(['I4', 'H4', 'V6', 'I6', 'V8', 'I3', 'I5', 'R2', 'W1', 'V1', 'H6',
       'I2', 'W8'], dtype=object)

In [45]:
df["engine"].unique()

array(['I4', 'H4', 'V6', 'I6', 'V8', 'I3', 'I5', 'R2', 'W1', 'V1', 'H6',
       'I2', 'W8'], dtype=object)

In [46]:
df.drop(columns=["engine_type"], inplace=True)

In [47]:
df.head()

,back_legroom,bed,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,exterior_color,fleet,...,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year,engine
0,35.09375,N/A,SUV / Crossover,Bayamon,21.629598,522,960,1300.0,Solar Yellow,NaN,...,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019,I4
1,38.09375,N/A,SUV / Crossover,San Juan,21.629598,207,922,2000.0,Narvik Black,NaN,...,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020,I4
2,35.40625,N/A,Sedan,Guaynabo,17.000000,1233,969,2500.0,None,False,...,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016,H4
3,37.59375,N/A,SUV / Crossover,San Juan,21.629598,196,922,3000.0,Eiger Gray,NaN,...,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020,V6
4,38.09375,N/A,SUV / Crossover,San Juan,21.629598,137,922,2000.0,Narvik Black,NaN,...,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020,I4


In [48]:
len(df[df.engine_displacement.isna()]["engine_displacement"])

97212

In [49]:
df[df.price.isna()]

,back_legroom,bed,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,exterior_color,fleet,...,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year,engine


In [50]:
df.loc[:, ["engine_displacement", "price"]].corr(method="pearson")

,engine_displacement,price
engine_displacement,1.000000,0.373978
price,0.373978,1.000000


In [51]:
df["exterior_color"].unique()

array(['Solar Yellow', 'Narvik Black', 'None', ..., 'Low Vol Bright Red',
       'Modern S', 'Cosmic'], dtype=object)

In [52]:
len(df["exterior_color"].unique())

27220

In [53]:
mask_color = df.exterior_color.value_counts() <= 20
mask_color.loc[mask_color.to_numpy()]

01g3/Fb20                    True
PACIFIC PEWTER METALLIC      True
01k3/Fb10                    True
Atlantic Blue Pearlcoat      True
Sage Brush Pearl             True
                             ... 
BLUE (DARK)                  True
RHINO CLEARCOAT              True
Ruby Red Metalic             True
Ebony Twighlight Metallic    True
Cosmic                       True
Name: exterior_color, Length: 23582, dtype: bool

In [54]:
len(df["dealer_zip"].unique())

9177

In [55]:
df["is_certified"].value_counts(dropna=False)

NaN    2891520
Name: is_certified, dtype: int64

In [56]:
df.drop(columns=["is_certified"], inplace=True)

In [57]:
df["owner_count"].fillna(value=0, inplace=True)

In [58]:
df.drop(columns=["power"], inplace=True)

In [59]:
df["horsepower"].value_counts(dropna=False)

170.0    120729
NaN       97212
375.0     78268
395.0     70141
180.0     69602
          ...  
376.0         1
77.0          1
331.0         1
326.0         1
69.0          1
Name: horsepower, Length: 456, dtype: int64

In [60]:
df["torque"].value_counts(dropna=False)

NaN                      442066
383 lb-ft @ 4,100 RPM     86137
400 lb-ft @ 4,500 RPM     63808
410 lb-ft @ 3,950 RPM     54227
148 lb-ft @ 200 RPM       53915
                          ...  
280 lb-ft @ 2,200 RPM         1
93 lb-ft @ 4,500 RPM          1
206 lb-ft @ 5,000 RPM         1
420 lb-ft @ 2,800 RPM         1
130 lb-ft @ 3,000 RPM         1
Name: torque, Length: 2054, dtype: int64

In [61]:
df.drop(columns=["torque"], inplace=True)
df.dropna(axis=0, subset="horsepower", inplace=True)

In [62]:
df.head()

,back_legroom,bed,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,exterior_color,fleet,...,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year,engine
0,35.09375,N/A,SUV / Crossover,Bayamon,21.629598,522,960,1300.0,Solar Yellow,NaN,...,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019,I4
1,38.09375,N/A,SUV / Crossover,San Juan,21.629598,207,922,2000.0,Narvik Black,NaN,...,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020,I4
2,35.40625,N/A,Sedan,Guaynabo,17.000000,1233,969,2500.0,None,False,...,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016,H4
3,37.59375,N/A,SUV / Crossover,San Juan,21.629598,196,922,3000.0,Eiger Gray,NaN,...,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020,V6
4,38.09375,N/A,SUV / Crossover,San Juan,21.629598,137,922,2000.0,Narvik Black,NaN,...,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020,I4


In [63]:
df.drop(columns=['exterior_color', 'interior_color', 'latitude', 'longitude'], inplace=True)

In [64]:
df["engine_type"] = df["engine"].apply(func=lambda x: re.split(r"(\d)", x)[0])
df["cylinder"] = df["engine"].apply(func=lambda x: re.split(r"(\d)", x)[1])
df.head()

,back_legroom,bed,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,...,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year,engine,engine_type,cylinder
0,35.09375,N/A,SUV / Crossover,Bayamon,21.629598,522,960,1300.0,NaN,NaN,...,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019,I4,I,4
1,38.09375,N/A,SUV / Crossover,San Juan,21.629598,207,922,2000.0,NaN,NaN,...,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020,I4,I,4
2,35.40625,N/A,Sedan,Guaynabo,17.000000,1233,969,2500.0,False,False,...,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016,H4,H,4
3,37.59375,N/A,SUV / Crossover,San Juan,21.629598,196,922,3000.0,NaN,NaN,...,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020,V6,V,6
4,38.09375,N/A,SUV / Crossover,San Juan,21.629598,137,922,2000.0,NaN,NaN,...,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020,I4,I,4


In [65]:
df.drop(columns=["trimId", "city", "bed", "engine"], inplace=True) # "city", "bed", "engine"

In [66]:
df.head()

,back_legroom,body_type,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,franchise_make,...,transmission_display,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year,engine_type,cylinder
0,35.09375,SUV / Crossover,21.629598,522,960,1300.0,NaN,NaN,True,Jeep,...,9-Speed Automatic Overdrive,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019,I,4
1,38.09375,SUV / Crossover,21.629598,207,922,2000.0,NaN,NaN,True,Land Rover,...,9-Speed Automatic Overdrive,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020,I,4
2,35.40625,Sedan,17.000000,1233,969,2500.0,False,False,True,FIAT,...,6-Speed Manual,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016,H,4
3,37.59375,SUV / Crossover,21.629598,196,922,3000.0,NaN,NaN,True,Land Rover,...,8-Speed Automatic Overdrive,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020,V,6
4,38.09375,SUV / Crossover,21.629598,137,922,2000.0,NaN,NaN,True,Land Rover,...,9-Speed Automatic Overdrive,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020,I,4


In [67]:
df.loc[:, ['front_legroom', 'fuel_tank_volume', 'fuel_type', 'salvage', 'has_accidents']]

,front_legroom,fuel_tank_volume,fuel_type,salvage,has_accidents
0,41.2 in,12.7 gal,Gasoline,NaN,NaN
1,39.1 in,17.7 gal,Gasoline,NaN,NaN
2,43.3 in,15.9 gal,Gasoline,False,False
3,39 in,23.5 gal,Gasoline,NaN,NaN
4,39.1 in,17.7 gal,Gasoline,NaN,NaN
...,...,...,...,...,...
3000034,42.9 in,21.1 gal,Gasoline,False,False
3000035,40.9 in,14.9 gal,Gasoline,False,False
3000036,41 in,19.4 gal,Gasoline,NaN,NaN
3000038,41.5 in,14.8 gal,Diesel,False,False


In [68]:
type(df["front_legroom"][0])

str

In [69]:
df["front_legroom"] = df["front_legroom"].str.replace(" in", "")
df["front_legroom"] = df["front_legroom"].str.replace("--", "0")
df["front_legroom"] = df["front_legroom"].astype("float")
df.loc[(df.front_legroom == 0), ("front_legroom")] = df.loc[(df.front_legroom != 0), ("front_legroom")].mean()

In [70]:
df["fuel_tank_volume"] = df["fuel_tank_volume"].str.replace(" gal", "")
df["fuel_tank_volume"] = df["fuel_tank_volume"].str.replace("--", "0")
df["fuel_tank_volume"] = df["fuel_tank_volume"].astype("float")
df.loc[(df.fuel_tank_volume==0), ("fuel_tank_volume")] = df.loc[(df.fuel_tank_volume!=0), ("fuel_tank_volume")].mean()

In [71]:
df["fuel_type"].value_counts(dropna=False)

Gasoline                  2513689
Flex Fuel Vehicle          143485
Hybrid                      73203
Diesel                      40651
Biodiesel                   23134
Compressed Natural Gas        140
Propane                         6
Name: fuel_type, dtype: int64

In [72]:
df.reset_index(drop=True, inplace=True)
df = enconding(df, "fuel_type")
df

,back_legroom,body_type,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,franchise_make,...,year,engine_type,cylinder,Biodiesel,Compressed Natural Gas,Diesel,Flex Fuel Vehicle,Gasoline,Hybrid,Propane
0,35.09375,SUV / Crossover,21.629598,522,960,1300.0,NaN,NaN,True,Jeep,...,2019,I,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,38.09375,SUV / Crossover,21.629598,207,922,2000.0,NaN,NaN,True,Land Rover,...,2020,I,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,35.40625,Sedan,17.000000,1233,969,2500.0,False,False,True,FIAT,...,2016,H,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,37.59375,SUV / Crossover,21.629598,196,922,3000.0,NaN,NaN,True,Land Rover,...,2020,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,38.09375,SUV / Crossover,21.629598,137,922,2000.0,NaN,NaN,True,Land Rover,...,2020,I,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794303,32.59375,Pickup Truck,18.000000,89,95482,3500.0,False,False,True,Ford,...,2017,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2794304,39.68750,SUV / Crossover,26.000000,16,94533,1500.0,True,False,False,NaN,...,2018,I,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2794305,38.40625,SUV / Crossover,18.000000,171,94591,3600.0,NaN,NaN,True,Chevrolet,...,2020,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2794306,35.00000,Sedan,30.000000,11,94533,2000.0,False,False,False,NaN,...,2017,I,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [73]:
print(df["has_accidents"].value_counts(dropna=False), "\n", df["salvage"].value_counts(dropna=False))

NaN      1305804
False    1258493
True      230011
Name: has_accidents, dtype: int64 
 False    1478544
NaN      1305804
True        9960
Name: salvage, dtype: int64


In [74]:
df.drop(columns=['salvage', 'has_accidents'], inplace=True)

In [75]:
df.loc[:, ('height', 'highway_fuel_economy', 'horsepower', 'isCab', 'is_cpo', 'is_new')]

,height,highway_fuel_economy,horsepower,isCab,is_cpo,is_new
0,66.5 in,NaN,177.0,NaN,NaN,True
1,68 in,NaN,246.0,NaN,NaN,True
2,58.1 in,23.0,305.0,False,NaN,False
3,73 in,NaN,340.0,NaN,NaN,True
4,68 in,NaN,246.0,NaN,NaN,True
...,...,...,...,...,...,...
2794303,70.6 in,23.0,278.0,False,NaN,False
2794304,65.4 in,32.0,170.0,True,NaN,False
2794305,70.7 in,27.0,310.0,NaN,NaN,True
2794306,55.7 in,40.0,180.0,False,NaN,False


In [76]:
len(df.loc[df.height=="--", ("height")])

403

In [77]:
df.drop(index=df.loc[df.height=="--", ("height")].index, inplace=True)

In [78]:
df["height"] = df["height"].str.replace(" in", "")
df["height"] = df["height"].astype("float")
df["height"]

0          66.5
1          68.0
2          58.1
3          73.0
4          68.0
           ... 
2794303    70.6
2794304    65.4
2794305    70.7
2794306    55.7
2794307    68.1
Name: height, Length: 2793905, dtype: float64

In [79]:
df["highway_fuel_economy"].value_counts(dropna=False)

NaN     313005
27.0    162923
28.0    160141
25.0    159443
30.0    156565
         ...  
58.0         9
55.0         8
10.0         5
72.0         4
56.0         1
Name: highway_fuel_economy, Length: 63, dtype: int64

In [80]:
df.dropna(axis=0, subset=["highway_fuel_economy"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,back_legroom,body_type,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,franchise_make,...,year,engine_type,cylinder,Biodiesel,Compressed Natural Gas,Diesel,Flex Fuel Vehicle,Gasoline,Hybrid,Propane
0,35.406250,Sedan,17.0,1233,969,2500.0,False,False,True,FIAT,...,2016,H,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,35.093750,Sedan,22.0,1233,969,3000.0,False,False,True,FIAT,...,2016,I,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38.093750,Sedan,27.0,55,11706,1500.0,True,False,True,Chevrolet,...,2018,I,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,27.296875,Coupe,18.0,36,10466,3500.0,False,False,True,Jeep,...,2018,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,38.406250,SUV / Crossover,18.0,27,11706,3600.0,False,False,True,Chevrolet,...,2018,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480895,32.593750,Pickup Truck,18.0,89,95482,3500.0,False,False,True,Ford,...,2017,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2480896,39.687500,SUV / Crossover,26.0,16,94533,1500.0,True,False,False,NaN,...,2018,I,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2480897,38.406250,SUV / Crossover,18.0,171,94591,3600.0,NaN,NaN,True,Chevrolet,...,2020,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2480898,35.000000,Sedan,30.0,11,94533,2000.0,False,False,False,NaN,...,2017,I,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [81]:
df["horsepower"].value_counts(dropna=False)

170.0     97540
375.0     74935
395.0     67836
180.0     65509
310.0     63820
          ...  
79.0          1
55.0          1
313.0         1
1001.0        1
69.0          1
Name: horsepower, Length: 440, dtype: int64

In [82]:
df[df.horsepower.isna()]

,back_legroom,body_type,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,franchise_make,...,year,engine_type,cylinder,Biodiesel,Compressed Natural Gas,Diesel,Flex Fuel Vehicle,Gasoline,Hybrid,Propane


In [83]:
df[df.horsepower.isna()]

,back_legroom,body_type,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,franchise_make,...,year,engine_type,cylinder,Biodiesel,Compressed Natural Gas,Diesel,Flex Fuel Vehicle,Gasoline,Hybrid,Propane


In [84]:
df["isCab"].value_counts(dropna=False)

NaN      1163341
False    1067218
True      250341
Name: isCab, dtype: int64

In [85]:
df.drop(columns=["isCab"], inplace=True)

In [86]:
# is_cpo	is_new
print("{0}\n{1}".format(df["is_cpo"].value_counts(dropna=False), df["is_new"].value_counts(dropna=False)))

NaN     2319574
True     161326
Name: is_cpo, dtype: int64
False    1280055
True     1200845
Name: is_new, dtype: int64


In [87]:
df.drop(columns=["is_cpo"], inplace=True)

In [88]:
df.loc[:, ['sp_name', 'theft_title', 'transmission']]

,sp_name,theft_title,transmission
0,FIAT de San Juan,False,M
1,FIAT de San Juan,False,A
2,Atlantic Chevrolet Cadillac,False,A
3,Eastchester Chrysler Jeep Dodge Ram,False,A
4,Atlantic Chevrolet Cadillac,False,A
...,...,...,...
2480895,Redwood Ford,False,A
2480896,CarMax Fairfield - Now offering Curbside Picku...,False,A
2480897,Team Chevrolet,NaN,A
2480898,CarMax Fairfield - Now offering Curbside Picku...,False,A


In [89]:
len(df["sp_name"].unique())

25941

In [90]:
df.drop(columns=["sp_name"], inplace=True)

In [91]:
print("{0}\n\n{1}".format(df["theft_title"].value_counts(dropna=False), df["transmission"].value_counts(dropna=False)))

False    1313167
NaN      1163341
True        4392
Name: theft_title, dtype: int64

A              1969414
CVT             433668
M                40396
NaN              28541
Dual Clutch       8881
Name: transmission, dtype: int64


In [92]:
df.drop(columns="theft_title", inplace=True)
df.dropna(subset=["transmission"], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [93]:
df

,back_legroom,body_type,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,franchise_make,...,year,engine_type,cylinder,Biodiesel,Compressed Natural Gas,Diesel,Flex Fuel Vehicle,Gasoline,Hybrid,Propane
0,35.406250,Sedan,17.0,1233,969,2500.0,False,False,True,FIAT,...,2016,H,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,35.093750,Sedan,22.0,1233,969,3000.0,False,False,True,FIAT,...,2016,I,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38.093750,Sedan,27.0,55,11706,1500.0,True,False,True,Chevrolet,...,2018,I,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,27.296875,Coupe,18.0,36,10466,3500.0,False,False,True,Jeep,...,2018,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,38.406250,SUV / Crossover,18.0,27,11706,3600.0,False,False,True,Chevrolet,...,2018,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452354,32.593750,Pickup Truck,18.0,89,95482,3500.0,False,False,True,Ford,...,2017,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2452355,39.687500,SUV / Crossover,26.0,16,94533,1500.0,True,False,False,NaN,...,2018,I,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2452356,38.406250,SUV / Crossover,18.0,171,94591,3600.0,NaN,NaN,True,Chevrolet,...,2020,V,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2452357,35.000000,Sedan,30.0,11,94533,2000.0,False,False,False,NaN,...,2017,I,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [94]:
df.loc[:, ['is_oemcpo', 'length', 'listed_date', 'listing_color', 'listing_id', 'main_picture_url']]

,is_oemcpo,length,listed_date,listing_color,listing_id,main_picture_url
0,NaN,180.9 in,2017-04-25,UNKNOWN,173473508,NaN
1,NaN,182.5 in,2017-04-25,SILVER,173473505,https://static.cargurus.com/images/forsale/201...
2,True,193.8 in,2020-07-16,SILVER,276675304,https://static.cargurus.com/images/forsale/202...
3,NaN,184.8 in,2020-08-04,BLACK,278352194,https://static.cargurus.com/images/forsale/202...
4,True,204.3 in,2020-08-13,SILVER,279129180,https://static.cargurus.com/images/forsale/202...
...,...,...,...,...,...,...
2452354,NaN,212.3 in,2020-06-15,WHITE,274134037,NaN
2452355,NaN,183.1 in,2020-08-27,SILVER,280498781,https://static.cargurus.com/images/forsale/202...
2452356,NaN,204.3 in,2020-03-25,BLACK,269431681,https://static.cargurus.com/images/forsale/202...
2452357,NaN,183.9 in,2020-09-01,GREEN,280923167,https://static.cargurus.com/images/forsale/202...


In [95]:
df["is_oemcpo"].value_counts(dropna=False)

NaN     2335178
True     117181
Name: is_oemcpo, dtype: int64

In [96]:
df["listing_color"].value_counts(dropna=False)

WHITE      507945
BLACK      487725
UNKNOWN    336646
SILVER     325595
GRAY       316410
BLUE       207140
RED        207014
GREEN       18903
BROWN       17676
ORANGE       9430
GOLD         8375
TEAL         4671
YELLOW       3510
PURPLE       1240
PINK           79
Name: listing_color, dtype: int64

In [97]:
df = enconding(df=df, column="listing_color")
df

,back_legroom,body_type,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,franchise_make,...,GREEN,ORANGE,PINK,PURPLE,RED,SILVER,TEAL,UNKNOWN,WHITE,YELLOW
0,35.406250,Sedan,17.0,1233,969,2500.0,False,False,True,FIAT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,35.093750,Sedan,22.0,1233,969,3000.0,False,False,True,FIAT,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,38.093750,Sedan,27.0,55,11706,1500.0,True,False,True,Chevrolet,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,27.296875,Coupe,18.0,36,10466,3500.0,False,False,True,Jeep,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.406250,SUV / Crossover,18.0,27,11706,3600.0,False,False,True,Chevrolet,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452354,32.593750,Pickup Truck,18.0,89,95482,3500.0,False,False,True,Ford,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2452355,39.687500,SUV / Crossover,26.0,16,94533,1500.0,True,False,False,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2452356,38.406250,SUV / Crossover,18.0,171,94591,3600.0,NaN,NaN,True,Chevrolet,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2452357,35.000000,Sedan,30.0,11,94533,2000.0,False,False,False,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
df.drop(index=df[df.length=="--"].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [99]:
df["length"] = df["length"].str.replace(pat=" in", repl="").astype("float")
df["length"]

0          180.9
1          182.5
2          193.8
3          184.8
4          204.3
           ...  
2452353    212.3
2452354    183.1
2452355    204.3
2452356    183.9
2452357    184.5
Name: length, Length: 2452358, dtype: float64

In [101]:
df.drop(columns=["listing_id", "main_picture_url", "is_oemcpo", "listed_date"], inplace=True)

In [102]:
df.loc[:, ['owner_count', 'price', 'savings_amount', 'seller_rating', 'sp_id']]

,owner_count,price,savings_amount,seller_rating,sp_id
0,3.0,46995.0,0,NaN,370467.0
1,2.0,58995.0,0,NaN,370467.0
2,1.0,14639.0,1749,3.447761,314501.0
3,1.0,32000.0,1861,2.800000,62178.0
4,1.0,23723.0,3500,3.447761,314501.0
...,...,...,...,...,...
2452353,1.0,40993.0,2220,5.000000,55297.0
2452354,1.0,17998.0,381,4.272727,341247.0
2452355,0.0,36490.0,0,4.533333,310674.0
2452356,1.0,26998.0,849,4.272727,341247.0


In [103]:
len(df[df.savings_amount.isna()])

0

In [104]:
len(df[df.seller_rating.isna()])

31556

In [105]:
df.dropna(subset="seller_rating", inplace=True)
df.reset_index(drop=True, inplace=True)

In [106]:
df.dropna(subset="fleet", inplace=True)
df.dropna(subset="frame_damaged", inplace=True)
df.dropna(subset="franchise_make", inplace=True)
df.reset_index(drop=True, inplace=True)

In [107]:
df.drop(columns=["sp_id"], inplace=True)

In [108]:
df

,back_legroom,body_type,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,franchise_make,...,GREEN,ORANGE,PINK,PURPLE,RED,SILVER,TEAL,UNKNOWN,WHITE,YELLOW
0,38.093750,Sedan,27.0,55,11706,1500.0,True,False,True,Chevrolet,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,27.296875,Coupe,18.0,36,10466,3500.0,False,False,True,Jeep,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38.406250,SUV / Crossover,18.0,27,11706,3600.0,False,False,True,Chevrolet,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,36.812500,SUV / Crossover,15.0,27,11706,3600.0,False,False,True,Chevrolet,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,38.593750,SUV / Crossover,18.0,24,11706,3600.0,False,False,True,Chevrolet,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834601,34.000000,Hatchback,35.0,59,94559,1400.0,False,False,True,Nissan,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834602,38.000000,SUV / Crossover,26.0,17,94559,2500.0,False,False,True,Nissan,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
834603,36.093750,Sedan,26.0,17,94559,2500.0,True,False,True,Nissan,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834604,32.593750,Pickup Truck,18.0,89,95482,3500.0,False,False,True,Ford,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [109]:
df = enconding(df=df, column="body_type")
df = enconding(df=df, column="franchise_make")
df

,back_legroom,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,front_legroom,fuel_tank_volume,...,RAM,Rolls-Royce,SRT,Scion,Shelby,Subaru,Toyota,Volkswagen,Volvo,smart
0,38.093750,27.0,55,11706,1500.0,True,False,True,42.0,15.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,27.296875,18.0,36,10466,3500.0,False,False,True,45.4,17.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38.406250,18.0,27,11706,3600.0,False,False,True,41.0,19.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,36.812500,15.0,27,11706,3600.0,False,False,True,41.3,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.593750,18.0,24,11706,3600.0,False,False,True,40.3,24.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834601,34.000000,35.0,59,94559,1400.0,False,False,True,42.0,9.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834602,38.000000,26.0,17,94559,2500.0,False,False,True,43.0,15.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834603,36.093750,26.0,17,94559,2500.0,True,False,True,45.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834604,32.593750,18.0,89,95482,3500.0,False,False,True,42.9,21.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
for s in df.columns:
    if type(df[s][0]) == str:
        print("{0}\t{1}\n".format(type(df[s][0]), s))

<class 'str'>	model_name



In [115]:
df["maximum_seating"].value_counts(dropna=False)
df.dropna(subset=["maximum_seating"], inplace=True)
df.reset_index(drop=True, inplace=True)
df["maximum_seating"] = df["maximum_seating"].str.replace(pat=" seats", repl="").astype("int8")
df["maximum_seating"]

0         5
1         4
2         8
3         8
4         5
         ..
834050    4
834051    5
834052    5
834053    5
834054    7
Name: maximum_seating, Length: 834055, dtype: int8

In [117]:
df["transmission"]
df = enconding(df=df, column="transmission")
df

,back_legroom,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,front_legroom,fuel_tank_volume,...,Shelby,Subaru,Toyota,Volkswagen,Volvo,smart,A,CVT,Dual Clutch,M
0,38.093750,27.0,55,11706,1500.0,True,False,True,42.0,15.8,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,27.296875,18.0,36,10466,3500.0,False,False,True,45.4,17.4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,38.406250,18.0,27,11706,3600.0,False,False,True,41.0,19.4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,36.812500,15.0,27,11706,3600.0,False,False,True,41.3,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,38.593750,18.0,24,11706,3600.0,False,False,True,40.3,24.6,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834050,34.000000,35.0,59,94559,1400.0,False,False,True,42.0,9.3,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
834051,38.000000,26.0,17,94559,2500.0,False,False,True,43.0,15.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
834052,36.093750,26.0,17,94559,2500.0,True,False,True,45.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
834053,32.593750,18.0,89,95482,3500.0,False,False,True,42.9,21.1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [118]:
df["transmission_display"].value_counts(dropna=False)

Automatic                             327161
6-Speed Automatic                     177584
Continuously Variable Transmission    134007
8-Speed Automatic                      94731
9-Speed Automatic                      35715
6-Speed Automatic Overdrive            14050
5-Speed Automatic                      13700
7-Speed Automatic                      10330
6-Speed Manual                          7548
4-Speed Automatic                       6923
Manual                                  5680
6-Speed Dual Clutch                     2157
5-Speed Manual                          1954
8-Speed Dual Clutch                      757
7-Speed CVT                              523
7-Speed Dual Clutch                      366
5-Speed Automatic Overdrive              236
7-Speed Manual                           163
10-Speed Automatic                       154
6-Speed CVT                              136
8-Speed CVT                               44
4-Speed Automatic Overdrive               34
8-Speed Au

In [120]:
df["wheel_system"].value_counts(dropna=False)
df.dropna(subset="wheel_system", inplace=True)
df = enconding(df=df, column="wheel_system")
df

,back_legroom,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,front_legroom,fuel_tank_volume,...,smart,A,CVT,Dual Clutch,M,4WD,4X2,AWD,FWD,RWD
0,38.093750,27.0,55,11706,1500.0,True,False,True,42.0,15.8,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,27.296875,18.0,36,10466,3500.0,False,False,True,45.4,17.4,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38.406250,18.0,27,11706,3600.0,False,False,True,41.0,19.4,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,36.812500,15.0,27,11706,3600.0,False,False,True,41.3,22.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,38.593750,18.0,24,11706,3600.0,False,False,True,40.3,24.6,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833972,34.000000,35.0,59,94559,1400.0,False,False,True,42.0,9.3,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
833973,38.000000,26.0,17,94559,2500.0,False,False,True,43.0,15.9,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
833974,36.093750,26.0,17,94559,2500.0,True,False,True,45.0,18.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
833975,32.593750,18.0,89,95482,3500.0,False,False,True,42.9,21.1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [121]:
df["wheelbase"]

0         111.4 in
1         107.5 in
2         120.9 in
3         118.9 in
4         114.8 in
            ...   
833972    105.7 in
833973    103.9 in
833974    109.3 in
833975    127.4 in
833976    106.5 in
Name: wheelbase, Length: 833977, dtype: object

In [122]:
df["width"].value_counts(dropna=False)

72.4 in    38686
96.8 in    31600
80 in      28250
73 in      26180
83.5 in    20478
           ...  
65.3 in        1
89.1 in        1
57 in          1
64 in          1
62.6 in        1
Name: width, Length: 251, dtype: int64

In [123]:
df = enconding(df=df, column="engine_type")
df["cylinder"] = df["cylinder"].astype("int8")
df

,back_legroom,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,fleet,frame_damaged,franchise_dealer,front_legroom,fuel_tank_volume,...,4WD,4X2,AWD,FWD,RWD,H,I,R,V,W
0,38.093750,27.0,55,11706,1500.0,True,False,True,42.0,15.8,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,27.296875,18.0,36,10466,3500.0,False,False,True,45.4,17.4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,38.406250,18.0,27,11706,3600.0,False,False,True,41.0,19.4,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,36.812500,15.0,27,11706,3600.0,False,False,True,41.3,22.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,38.593750,18.0,24,11706,3600.0,False,False,True,40.3,24.6,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833972,34.000000,35.0,59,94559,1400.0,False,False,True,42.0,9.3,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
833973,38.000000,26.0,17,94559,2500.0,False,False,True,43.0,15.9,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
833974,36.093750,26.0,17,94559,2500.0,True,False,True,45.0,18.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
833975,32.593750,18.0,89,95482,3500.0,False,False,True,42.9,21.1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [124]:
df.drop(columns=["major_options", "make_name", "trim_name", "transmission_display", "wheel_system_display", "wheelbase", "width", "model_name"],
        inplace=True)

In [133]:
for s in df.columns:
    if type(df[s][0]) == bool or type(df[s][0]) == np.bool_:
        print("{0}\t{1}\n".format(type(df[s][0]), s))

<class 'bool'>	fleet

<class 'bool'>	frame_damaged

<class 'numpy.bool_'>	franchise_dealer

<class 'numpy.bool_'>	is_new



In [134]:
print("{0}\n\n{1}\n\n{2}\n\n{3}"
      .format(df["fleet"].value_counts(dropna=False), df["frame_damaged"].value_counts(dropna=False),
              df["franchise_dealer"].value_counts(dropna=False), df["is_new"].value_counts(dropna=False), ))

False    677530
True     156447
Name: fleet, dtype: int64

False    832250
True       1727
Name: frame_damaged, dtype: int64

True    833977
Name: franchise_dealer, dtype: int64

False    795791
True      38186
Name: is_new, dtype: int64


In [135]:
df["fleet"] = [1 if x else 0 for x in df["fleet"]]
df["frame_damaged"] = [1 if x else 0 for x in df["frame_damaged"]]
df["franchise_dealer"] = [1 if x else 0 for x in df["franchise_dealer"]]
df["is_new"] = [1 if x else 0 for x in df["is_new"]]

In [137]:
len(df.columns)

115

In [139]:
X = df.drop(columns=["price"])
y = df["price"]
y

0         14639.0
1         32000.0
2         23723.0
3         22422.0
4         29424.0
           ...   
833972    10900.0
833973    20700.0
833974    17700.0
833975    40993.0
833976    19900.0
Name: price, Length: 833977, dtype: float64

In [141]:
X.shape

(833977, 114)

In [144]:
price_model = tf.keras.Sequential([
    tf.keras.layers.Input(X.shape),
    tf.keras.layers.Dense(114),
    tf.keras.layers.Dense(57),
    tf.keras.layers.Dense(20),
    tf.keras.layers.Dense(1),
])

price_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error')

In [149]:
X_train, X_test, y_train, y_test= train_test_split(X, y, train_size=0.8, test_size=0.2)

In [151]:
df = 1

In [155]:
price_model.fit(X_train, y_train)

MemoryError: Unable to allocate 580. MiB for an array with shape (114, 667181) and data type object